In [ ]:
# %% [code]
# ---------- 导入库 ----------
import csv
import sqlite3
import re
from rich.console import Console
from rich.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import calendar
from tabulate import tabulate
import os

In [ ]:
# %% [code]
# ---------- 全局参数设置 ----------
# 设置当前处理的月份（格式：YYYYMM）
CURRENT_MONTH = "202506"

# 数据库路径
DB_PATH = "moneycount.db"

# 标签映射配置
TAG_MAPPING = {
    "外卖": "eat_ol",
    "早餐": "eat_bf",
    "饮料": "eat_dk",
    "出租车": "traffic_dd",
    "地铁": "traffic_ud",
    "充电宝": "traffic_pow",
    "铁路": "tour_high",
    "酒店": "tour_hotel",
    "单车": "traffic_bike",
    "手机": "olpay_phone",
    "软件": "olpay_soft",
    "聚餐": "social_eat",
    "礼物": "social_buy",
    "游戏": "olpay_game",
    "电器": "buy_elec",
    "衣服": "buy_cloth",
    "洗漱": "buy_wash",
    "书": "buy_book",
    "医疗": "buy_medical",
    "生活": "living_use",
    "剪发": "living_hair",
    "快递": "living_delivery",
    "工资": "income_salary",
    "副业": "income_sideline",
    "投资": "income_invest",
    "考试": "invest_exam",
    "上课": "invest_course",
    "房租": "house_rent",
    "会员": "olpay_vip",
    "午餐": "eat_m",
    "红包": "social_hong",
    "娱乐": "social_fun",
    "转账": "social_ex",
    "信用卡": "credit"
}

# 常用支付源到标签的自动映射（可根据历史记录习惯更新）
AUTO_TAG_MAPPING = {
    "钱大妈": "午餐",
    "肯德基": "午餐",
    "美团": "午餐",
    "地铁": "地铁",
    "岭南通": "地铁",
    "铁路": "铁路",
    "滴滴": "出租车",
    "手机": "手机",
    "便利": "饮料",
    "美宜佳": "饮料",
    "零食": "饮料",
    "蜜雪": "饮料",
    "Apple": "会员",
    "转账": "转账",
    "收款": "转账",
    "发给": "红包",
    "房租": "房租",
    "优剪": "剪发",
    "医疗": "医疗",
    "医院": "医疗",
    "铁臂": "会员",
    "丰巢": "快递",
    "顺丰": "快递",
    "京东": "生活",
    "阿里": "软件",
    "百度": "会员",
    "Steam": "游戏"
}

In [ ]:
# %% [code]
# ---------- SQL语句定义 ----------
def sql_create_scheme_pay_monthyear(table_name: str):
    """创建支付记录表的SQL语句"""
    return f"""
    CREATE TABLE IF NOT EXISTS {table_name}  (
      id BIGINT NOT NULL,
      pay_time VARCHAR NOT NULL,
      pay_monthyear VARCHAR NOT NULL,
      pay_source VARCHAR,
      pay_note VARCHAR,
      pay_money NUMERIC NOT NULL,
      pay_tag VARCHAR,
      app_source VARCHAR,
      PRIMARY KEY (id, pay_monthyear, pay_money)
    )
    """

In [ ]:
# %% [code]
# ---------- 工具函数 ----------
def sanitize_column_name(name):
    """清理列名，确保符合SQLite标识符规范"""
    cleaned_name = re.sub(r'\W+', '', name)
    if cleaned_name and cleaned_name[0].isdigit():
        cleaned_name = 'col_' + cleaned_name
    return cleaned_name.lower()

def get_column_type(column_name):
    """根据列名确定数据类型"""
    money_keywords = ['金额', '退款', '价格', '费用', '余额', '服务费']
    if any(keyword in column_name for keyword in money_keywords):
        return 'REAL'
    return 'TEXT'

def is_money_column(column_name):
    """判断是否为金额列"""
    money_keywords = ['金额', '退款', '价格', '费用', '余额', '服务费']
    return any(keyword in column_name for keyword in money_keywords)

def clean_money_value(value):
    """清理金额数据"""
    if not value:
        return 0.0
    cleaned_value = value.replace('¥', '').replace(',', '').strip()
    try:
        return float(cleaned_value)
    except ValueError:
        return cleaned_value

def rename_existing_table(cursor, table_name):
    """检查表是否存在，如果存在则将其重命名"""
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if cursor.fetchone():
            new_table_name = f"{table_name}_"
            while True:
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (new_table_name,))
                if not cursor.fetchone():
                    break
                new_table_name += "_"
            return True
        return False
    except Exception as e:
        print(f"重命名表时发生错误: {e}")
        return False

def get_current_table_name():
    """获取当前月份对应的表名"""
    return f"pay_{CURRENT_MONTH}"

def get_month_date_range(monthyear):
    """获取指定月份的起止日期"""
    year = int(monthyear[:4])
    month = int(monthyear[4:])
    _, last_day = calendar.monthrange(year, month)
    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month:02d}-{last_day}"
    return start_date, end_date

In [ ]:
# %% [code]
# ---------- CSV导入功能 ----------
def import_csv_to_sqlite(csv_file_path, strategy, header_row=0, start_row=0, skip_columns=None, params=None):
    """
    导入CSV文件到SQLite数据库
    :param csv_file_path: CSV文件路径
    :param strategy: 导入策略 ('wx', 'zfb', 'yh', 'other')
    :param header_row: 表头所在的行号（从0开始）
    :param start_row: 数据开始的行号（从0开始）
    :param skip_columns: 要跳过的列索引列表
    :param params: 策略所需的额外参数
    """
    if skip_columns is None:
        skip_columns = []
    if params is None:
        params = {}
    
    # 添加月份参数
    params['monthyear'] = CURRENT_MONTH
    table_name = get_current_table_name()

    try:
        encodings = ['gb2312', 'gbk', 'gb18030', 'utf-8']
        
        for encoding in encodings:
            try:
                conn = sqlite3.connect(DB_PATH)
                cursor = conn.cursor()

                # 首先导入CSV到临时表
                with open(csv_file_path, 'r', encoding=encoding) as csvfile:
                    csv_reader = csv.reader(csvfile)

                    # 跳过header_row之前的行
                    for _ in range(header_row):
                        next(csv_reader)

                    # 读取表头行
                    headers = next(csv_reader)

                    # 过滤掉要跳过的列
                    filtered_headers = [h for i, h in enumerate(headers) if i not in skip_columns]
                    cleaned_headers = [sanitize_column_name(header) for header in filtered_headers]

                    # 创建临时表
                    strategy_table_name = f"t_{table_name}_{strategy}"

                    # 重命名已存在的表或创建新表
                    if rename_existing_table(cursor, strategy_table_name):
                        print(f"表 {strategy_table_name} 已存在并已重命名")
                    
                    # 创建临时表
                    create_strategy_table_query = f"""
                        CREATE TABLE IF NOT EXISTS {strategy_table_name} (
                            {', '.join([f'[{col}] {get_column_type(col)}' for col in cleaned_headers])}
                        )
                    """
                    cursor.execute(create_strategy_table_query)

                    # 跳过start_row之前的行
                    remaining_rows_to_skip = start_row - (header_row + 1)
                    if remaining_rows_to_skip > 0:
                        for _ in range(remaining_rows_to_skip):
                            next(csv_reader)

                    # 插入数据到临时表
                    for row in csv_reader:
                        if not row:  # 跳过空行
                            continue

                        filtered_row = []
                        for i, value in enumerate(row):
                            if i not in skip_columns:
                                if i < len(cleaned_headers) and is_money_column(cleaned_headers[i]):
                                    filtered_row.append(clean_money_value(value))
                                else:
                                    filtered_row.append(value)

                        if len(filtered_row) == len(cleaned_headers):
                            placeholders = ','.join(['?' for _ in filtered_row])
                            insert_query = f"INSERT INTO {strategy_table_name} VALUES ({placeholders})"
                            try:
                                cursor.execute(insert_query, filtered_row)
                            except Exception as e:
                                print(f"插入失败: {e}")

                # 创建正式表
                create_table_query = sql_create_scheme_pay_monthyear(table_name)
                cursor.execute(create_table_query)

                # 根据策略执行相应的SQL将数据导入正式表
                if strategy == 'wx':
                    sql1 = f"""
                    INSERT INTO [{table_name}] (id, pay_time, pay_monthyear, pay_source, pay_note, pay_money, pay_tag, app_source)
                         SELECT
                         strftime('%s',[交易时间]) as id,
                         [交易时间] as pay_time,
                         '{params.get('monthyear')}' as pay_monthyear,
                         [交易对方] as pay_source,
                         [商品] as pay_note,
                         cast([金额元] as real)  as pay_money,
                         'N' as 'pay_tag',
                         'wx' as 'app_source'
                         FROM '{strategy_table_name}' where [收支]='支出'
                    """
                    sql2 = f"""
                    INSERT INTO [{table_name}] (id, pay_time, pay_monthyear, pay_source, pay_note, pay_money, pay_tag, app_source)
                         SELECT
                         strftime('%s',[交易时间]) as id,
                         [交易时间] as pay_time,
                         '{params.get('monthyear')}' as pay_monthyear,
                         [交易对方] as pay_source,
                         ([商品] || [当前状态]) as pay_note,
                         cast(-[金额元] as real)  as pay_money, 
                         'N' as 'pay_tag',
                         'wx' as 'app_source'
                         FROM '{strategy_table_name}' where [收支]='收入'
                    """
                    cursor.execute(sql1)
                    cursor.execute(sql2)

                elif strategy == 'zfb':
                    sql3 = f"""
                    INSERT INTO [{table_name}] (id, pay_time, pay_monthyear, pay_source, pay_note, pay_money, pay_tag, app_source) 
                         SELECT
                         strftime('%s',[交易创建时间]) as id,
                         [交易创建时间] as pay_time,
                         '{params.get('monthyear')}' as pay_monthyear,
                         [交易对方] as pay_source,
                         [商品名称] as pay_note,
                         [金额元] as pay_money,
                         'N' as 'pay_tag',
                         'ali' as 'app_source'
                         FROM '{strategy_table_name}'  where [收支] like '支出%'  and [交易状态] like '交易成功%' and [成功退款元]=0
                    """
                    sql4 = f"""
                    INSERT INTO [{table_name}] (id, pay_time, pay_monthyear, pay_source, pay_note, pay_money, pay_tag, app_source) 
                         SELECT
                         strftime('%s',[交易创建时间]) as id,
                         [交易创建时间] as pay_time,
                         '{params.get('monthyear')}' as pay_monthyear,
                         [交易对方] as pay_source, 
                         [商品名称] as pay_note, 
                         [金额元]-[成功退款元]  as pay_money,
                         'N' as 'pay_tag',
                         'ali' as 'app_source'
                         FROM '{strategy_table_name}'  where  [交易状态] like '交易成功%' and [成功退款元]>0
                    """
                    cursor.execute(sql3)
                    cursor.execute(sql4)

                elif strategy == 'yh':
                    # 银行信用卡策略SQL
                    print("银行信用卡导入策略尚未实现")

                elif strategy == 'other':
                    # 其他SQL
                    print("其他导入策略尚未实现")

                # 提交事务
                conn.commit()
                print(f"成功使用 {encoding} 编码导入CSV并执行{strategy}策略")
                return

            except UnicodeDecodeError:
                conn.close()
                continue
            except sqlite3.Error as e:
                print(f"SQLite错误: {e}")
                conn.close()

        print("无法使用指定的任何编码解码文件")

    except Exception as e:
        print(f"发生错误: {e}")
    finally:
        if 'conn' in locals():
            conn.close()

In [ ]:
# %% [code]
# ---------- 查询功能 ----------
def query_payment_data(params):
    """
    查询支付数据
    :param params: 查询参数字典
    :return: (success, result)
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        table_name = get_current_table_name()
        
        # 检查表是否存在
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 构建查询条件
        conditions = []
        query_params = []
        
        # 标签筛选
        if params.get('tag'):
            conditions.append("pay_tag = ?")
            query_params.append(params['tag'])
        
        # 关键词搜索
        if params.get('key'):
            conditions.append("(pay_source LIKE ? OR pay_note LIKE ?)")
            query_params.extend([f"%{params['key']}%", f"%{params['key']}%"])
        
        # 金额范围筛选
        if params.get('lt'):
            try:
                conditions.append("pay_money < ?")
                query_params.append(float(params['lt']))
            except ValueError:
                return False, "金额上限格式错误"
        
        if params.get('gt'):
            try:
                conditions.append("pay_money > ?")
                query_params.append(float(params['gt']))
            except ValueError:
                return False, "金额下限格式错误"
        
        # 构建完整的SQL查询
        query = f"SELECT * FROM {table_name}"
        if conditions:
            query += " WHERE " + " AND ".join(conditions)
        query += " ORDER BY pay_money DESC"
        
        # 执行查询
        cursor.execute(query, query_params)
        
        # 获取列名
        columns = [description[0] for description in cursor.description]
        
        # 将结果转换为字典列表
        results = []
        for row in cursor.fetchall():
            results.append(dict(zip(columns, row)))
        
        return True, results
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"
    finally:
        if 'conn' in locals():
            conn.close()

def display_query_results(results, params):
    """
    使用rich库显示查询结果
    """
    console = Console()
    
    if not results:
        console.print("[red]没有找到数据[/red]")
        return
    
    # 创建表格
    table = Table(
        title=f"支付记录 ({CURRENT_MONTH})",
        show_header=True,
        header_style="bold magenta",
        border_style="blue",
        title_style="bold cyan"
    )
    
    # 添加列
    table.add_column("日期", style="cyan")
    table.add_column("金额", justify="right", style="green")
    table.add_column("来源", style="yellow")
    table.add_column("标签", style="red")
    table.add_column("备注")
    
    # 添加数据行
    for row in results:
        money = f"¥{float(row['pay_money']):.2f}"
        money_style = "[bold red]" + money + "[/bold red]" if float(row['pay_money']) > 100 else "[green]" + money + "[/green]"
            
        table.add_row(
            row['pay_time'],
            money_style,
            row['pay_source'],
            row['pay_tag'],
            row.get('pay_note', '')
        )
    
    # 显示查询条件
    console.print("\n[bold yellow]查询条件:[/bold yellow]")
    conditions = []
    if params.get('tag'):
        conditions.append(f"标签: {params['tag']}")
    if params.get('key'):
        conditions.append(f"关键词: {params['key']}")
    if params.get('lt'):
        conditions.append(f"金额 < {params['lt']}")
    if params.get('gt'):
        conditions.append(f"金额 > {params['gt']}")
    
    if conditions:
        console.print(" | ".join(conditions))
    
    # 打印表格
    console.print("\n")
    console.print(table)
    
    # 打印统计信息
    total_amount = sum(float(row['pay_money']) for row in results)
    console.print(f"\n[bold green]总金额: ¥{total_amount:.2f}[/bold green]")
    console.print(f"[dim]共 {len(results)} 条记录[/dim]")

In [ ]:
# %% [code]
# ---------- 更新功能 ----------
def update_payment_tags(params):
    """
    批量更新支付标签
    :param params: 更新参数字典
    :return: (success, message)
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        table_name = get_current_table_name()
        
        # 检查表是否存在
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 检查标签词是否在映射中
        tag_word = params.get('tag_word')
        if tag_word not in TAG_MAPPING:
            return False, f"不支持的标签词: {tag_word}"
            
        new_tag = TAG_MAPPING[tag_word]
        
        # 构建查询条件
        conditions = []
        query_params = []
        
        # 关键词搜索
        if params.get('key'):
            conditions.append("(pay_source LIKE ? OR pay_note LIKE ?)")
            query_params.extend([f"%{params['key']}%", f"%{params['key']}%"])
        
        # 金额范围筛选
        if params.get('lt'):
            try:
                conditions.append("pay_money < ?")
                query_params.append(float(params['lt']))
            except ValueError:
                return False, "金额上限格式错误"
        
        if params.get('gt'):
            try:
                conditions.append("pay_money > ?")
                query_params.append(float(params['gt']))
            except ValueError:
                return False, "金额下限格式错误"
        
        # 构建更新语句
        update_sql = f"UPDATE {table_name} SET pay_tag = ?"
        if conditions:
            update_sql += " WHERE " + " AND ".join(conditions)
            
        # 组合参数
        update_params = [new_tag] + query_params
        
        # 执行更新
        cursor.execute(update_sql, update_params)
        
        # 获取更新的行数
        rows_affected = cursor.rowcount
        
        # 提交事务
        conn.commit()
        return True, f"成功更新 {rows_affected} 条记录的标签为 {new_tag}"
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"
    finally:
        if 'conn' in locals():
            conn.close()

def auto_update_tags_based_on_history():
    """根据历史记录习惯自动更新标签"""
    results = []
    for source, tag in AUTO_TAG_MAPPING.items():
        params = {
            "key": source,
            "tag_word": tag
        }
        success, message = update_payment_tags(params)
        results.append(f"{source} -> {tag}: {message}")
    return results

In [ ]:
# %% [code]
# ---------- 图表分析功能 ----------
def query_monthly_data():
    """查询月度数据"""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        table_name = get_current_table_name()
        
        # 检查表是否存在
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 查询所有数据
        query = f"SELECT pay_time, pay_money, pay_tag FROM {table_name}"
        cursor.execute(query)
        
        # 转换为DataFrame
        df = pd.DataFrame(cursor.fetchall(), columns=['pay_time', 'pay_money', 'pay_tag'])
        
        # 处理日期格式
        df['pay_time'] = pd.to_datetime(df['pay_time'], errors='coerce', format='mixed')
        
        # 处理可能的NaT值
        df = df.dropna(subset=['pay_time'])
        
        return True, df
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"
    finally:
        if 'conn' in locals():
            conn.close()

In [ ]:
def plot_monthly_charts(df):
    """绘制月度图表"""
    # 设置字体
    plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans', 'sans-serif']
    plt.rcParams['axes.unicode_minus'] = False
    
    # 创建一个图形，包含两个子图
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. 饼图 - 按标签统计
    tag_sum = df.groupby('pay_tag')['pay_money'].sum()
    sorted_tags = sorted(tag_sum.index)
    tag_sum = tag_sum[sorted_tags]
    total = tag_sum.sum()
    percentages = tag_sum / total * 100
    
    labels = [f'{tag}\n¥{amount:.2f}\n({percent:.1f}%)' 
              for tag, amount, percent in zip(tag_sum.index, tag_sum, percentages)]
    
    ax1.pie(tag_sum, labels=labels, autopct='', startangle=90)
    ax1.set_title(f'{CURRENT_MONTH[:4]}年{CURRENT_MONTH[4:]}月消费类别分布')
    
    # 2. 柱状图 - 按日期统计
    start_date, end_date = get_month_date_range(CURRENT_MONTH)
    date_range = pd.date_range(start=start_date, end=end_date)
    
    # 按日期分组求和
    daily_sum = df.groupby('pay_time')['pay_money'].sum()
    
    # 创建完整的日期序列，填充缺失值为0
    daily_sum_full = pd.Series(0, index=date_range)
    daily_sum_full.update(daily_sum)
    
    # 绘制柱状图
    bars = ax2.bar(daily_sum.index.strftime('%d'), daily_sum.values)
    ax2.set_title(f'{CURRENT_MONTH[:4]}年{CURRENT_MONTH[4:]}月每日消费金额')
    ax2.set_xlabel('日期')
    ax2.set_ylabel('金额 (元)')
    plt.xticks(rotation=45)
    
    # 在每个柱子上方显示金额
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax2.text(
                bar.get_x() + bar.get_width()/2., 
                height + 0.1,
                f'¥{height:.0f}',
                ha='center',
                va='bottom',
                fontsize=8
            )
    
    plt.tight_layout()
    return fig

In [ ]:
# %% [code]
# ---------- OCR导入功能（DeepSeek OCR）----------
def import_ocr_bank_statement(image_path):
    """
    使用DeepSeek OCR处理银行截图并生成CSV
    注意：这里需要实际的OCR实现，此处为伪代码
    """
    print(f"使用DeepSeek OCR处理银行截图: {image_path}")
    # 伪代码 - 实际实现需要调用DeepSeek OCR API
    # result = deepseek_ocr.process(image_path)
    # csv_path = os.path.splitext(image_path)[0] + ".csv"
    # result.to_csv(csv_path, index=False)
    # return csv_path
    
    # 返回示例CSV路径（实际使用时应替换为真实路径）
    return "bank_statement.csv"

def import_bank_csv_to_db(csv_path, strategy='yh'):
    """导入银行CSV到数据库"""
    print(f"导入银行CSV: {csv_path}")
    
    # 银行CSV导入参数（需要根据实际CSV格式调整）
    params = {
        'header_row': 0,
        'start_row': 1,
        'skip_columns': [],
        'params': {'monthyear': CURRENT_MONTH}
    }
    
    import_csv_to_sqlite(csv_path, strategy, **params)

In [ ]:
# %% [code]
# ---------- 示例使用 ----------
if __name__ == "__main__":
    # 设置当前月份
    CURRENT_MONTH = "202506"
    
    # 动态生成文件路径
    def get_data_path(source, month=CURRENT_MONTH):
        """根据数据源和月份生成文件路径"""
        return f"data/{source}/{month}.csv"
    
    # 模式选择
    # 扩展模式选择
    def select_modes():
        """让用户选择要执行的功能模块"""
        print("\n请选择要执行的功能（可多选，用逗号分隔）:")
        print("1. import - 导入数据")
        print("2. auto_update - 自动更新标签")
        print("3. manual_update - 手动更新标签")  # 新增手动更新选项
        print("4. query - 查询数据")
        print("5. chart - 绘制图表")
        print("6. ocr - 处理银行对账单")
        
        choices = input("请输入选择（如：1,3,4）: ").strip().split(',')
        modes = []
        for choice in choices:
            choice = choice.strip()
            if choice == '1':
                modes.append('import')
            elif choice == '2':
                modes.append('auto_update')
            elif choice == '3':  # 新增手动更新
                modes.append('manual_update')
            elif choice == '4':
                modes.append('query')
            elif choice == '5':
                modes.append('chart')
            elif choice == '6':
                modes.append('ocr')
        return modes

    
    # 获取用户选择
    selected_modes = select_modes()
    if not selected_modes:
        print("未选择任何功能，程序退出")
        exit()
    
    print(f"\n当前处理月份: {CURRENT_MONTH}")
    print(f"将执行的功能: {', '.join(selected_modes)}")
    
    # 执行选中的功能
    if 'import' in selected_modes:
        print("\n=== 导入数据 ===")
        # 导入微信数据
        print("\n导入微信数据...")
        import_csv_to_sqlite(
            csv_file_path=get_data_path('wx'),
            strategy='wx',
            header_row=16,
            start_row=17,
            skip_columns=[8, 9]
        )
        
        # 导入支付宝数据
        print("\n导入支付宝数据...")
        import_csv_to_sqlite(
            csv_file_path=get_data_path('zfb'),
            strategy='zfb',
            header_row=4,
            start_row=5,
            skip_columns=[0, 1, 3, 16]
        )
    
    if 'update' in selected_modes:
        print("\n=== 自动更新标签 ===")
        update_results = auto_update_tags_based_on_history()
        for result in update_results:
            print(result)

    if 'manual_update' in selected_modes:
        print("\n=== 手动更新标签 ===")
        up_params = {
            "monthyear": CURRENT_MONTH,
            "key": input("输入要更新的关键词（如'肯德基'）: ").strip(),
            "tag_word": input("输入标签词（如'午餐'）: ").strip(),
            "lt": input("输入最大金额（留空跳过）: ").strip() or "",
            "gt": input("输入最小金额（留空跳过）: ").strip() or ""
        }
        
        # 检查标签词是否有效
        if up_params["tag_word"] not in TAG_MAPPING:
            print(f"错误: 无效的标签词 '{up_params['tag_word']}'")
            print("可用标签词:", ", ".join(TAG_MAPPING.keys()))
        else:
            success, message = update_payment_tags(up_params)
            print("\n" + message)
    
    if 'query' in selected_modes:
        print("\n=== 查询数据 ===")
        query_params = {
            "tag": input("输入标签筛选（留空跳过）: ").strip() or "",
            "key": input("输入关键词筛选（留空跳过）: ").strip() or "",
            "lt": input("输入最大金额（留空跳过）: ").strip() or "",
            "gt": input("输入最小金额（留空跳过）: ").strip() or ""
        }
        success, results = query_payment_data(query_params)
        if success:
            display_query_results(results, query_params)
        else:
            print(f"查询失败: {results}")
    
    if 'chart' in selected_modes:
        print("\n=== 绘制图表 ===")
        success, df = query_monthly_data()
        if success:
            fig = plot_monthly_charts(df)
            plt.show()
            
            # 输出统计信息
            print("\n统计信息:")
            print(f"总消费: ¥{df['pay_money'].sum():.2f}")
            print(f"平均每日消费: ¥{df['pay_money'].sum() / len(df['pay_money']):.2f}")
            print("\n按标签统计:")
            tag_stats = df.groupby('pay_tag').agg({
                'pay_money': ['sum', 'count']
            })
            print(tag_stats)
        else:
            print(f"数据查询失败: {df}")
    
    if 'ocr' in selected_modes:
        print("\n=== 处理银行对账单 ===")
        bank_image = input("请输入银行对账单图片路径: ").strip()
        if bank_image:
            csv_path = import_ocr_bank_statement(bank_image)
            import_bank_csv_to_db(csv_path)
        else:
            print("未提供图片路径，跳过OCR处理")
    
    print("\n所有选定功能执行完成！")